In [ ]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [ ]:
from lib import *

In [ ]:
import cdsapi
# import ddsapi

In [ ]:
output_path = "/media/lacie2022/data/meteo/ecmwf/era5/original/ensemble/"
mkNestedDir( output_path )

lat_min = 43
lat_max = 49
lon_min = 5
lon_max = 17

years = [
    # 2010,
    # 2011,
    # 2012,
    # 2013,
    # 2014,
    # 2015,
    # 2016,
    # 2017,
    # 2018,
    # 2019,
    # 2020,
    2021,
    2022
]

variables = ['total_precipitation']

runs = [
    "00:00",
    "03:00",
    "06:00",
    "09:00",
    "12:00",
    "15:00",
    "18:00",
    "21:00"
]

In [ ]:
os.chdir( output_path )

In [ ]:
json_conf = {
    'product_type': 'ensemble_members',
    'format': 'grib'
}

json_conf['area'] = [ lat_max, lon_min, lat_min, lon_max]

In [ ]:
def run_download(c_json, var, run, day, month, year):

    c_json['variable'] = var
    c_json['time'] = run
    c_json['day'] = day
    c_json['month'] = month
    c_json['year'] = year

    print(c_json)

    c = cdsapi.Client()

    c.retrieve(
        'reanalysis-era5-single-levels',
        c_json,
        '''{var}_{run}_{year}{month}{day}.grib'''.format(
            var=var,
            run=run[0:2],
            year=year,
            month=month.zfill(2),
            day=day.zfill(2)
        ))

In [ ]:
def execute(date):
    for run in runs:
        for var in variables:
            run_download(json_conf.copy(), var, run, str(date.day), str(date.month), str(date.year) )

In [ ]:
import multiprocessing

for year in years:
    start_date = dt.datetime.strptime( str(year)+'-01-01', '%Y-%m-%d' )
    end_date = dt.datetime.strptime( str(year)+'-12-31', '%Y-%m-%d' )
    dates = pd.date_range(start_date, end_date, freq='d')
    
    with multiprocessing.Pool() as pool:
        pool.map(execute, dates)
        
    # close the process pool
    pool.close()